###### Untuk training intent- agar bisa mengenali dari file input.txt
menjadi sebuah intent yang telah di encoding dengan menggunakan
susunan directory dalam 'data'

In [1]:
#!pip install gensim
#!pip install keras

In [2]:
import codecs
import glob
import re
import nltk
import gensim.models.word2vec as w2v
import numpy as np
import matplotlib.pyplot as plt
import os
import gensim
from keras.models import Sequential
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from keras import optimizers

C:\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# PARAMETERS

In [3]:
w2vmodel="./model/chat_model1-300.w2v"   #filename
num_features = 300    #number of features in the model
maxsent = 10          #maximum words in a sentence.

### LOAD VECTOR MODEL 

In [4]:
tex2vec =  gensim.models.KeyedVectors.load(w2vmodel)

In [5]:
from chat_wordvec import *

### Cek data anda

swv_ar(twl("irza ada disini")).shape

In [6]:
#test = swv_ar(twl("Saya tidak peduli. Itu terjadi menjadi nama saya.", rempunct=True, flat=True),frontpad=False)
#print(test.shape)
#print(test)

### LOAD DATA ANDA

In [7]:
import glob
import fileinput
fls = glob.glob('./data/*/input.txt')

In [8]:
intent=[]
prep_x = []
prep_y = []
for i in fls:
    for tx in fileinput.input(i):
        print(tx.replace('\n',''), '-->', i.split('\\')[-2])
        prep_x.append(tx.replace('\n',''))
        prep_y.append(i.split('\\')[-2])
        intent.append(i.split('\\')[-2])
fileinput.close()

intent = list(set(intent))
intent.sort()
print(intent)

ya --> affirmasi
iya --> affirmasi
betul --> affirmasi
yes --> affirmasi
setuju --> affirmasi
oke --> affirmasi
terima kasih --> affirmasi
iya --> affirmasi
terima kasih --> affirmasi
ho oh --> affirmasi
laksanakan --> affirmasi
asik --> affirmasi
benar --> affirmasi
kamu bobo saja --> awake
kamu bobok aja --> awake
kamu bobo aja --> awake
kamu bobok saja --> awake
kamu tidur saja --> awake
bobo --> awake
silahkan tidur --> awake
bangun --> awake
bangun sekarang --> awake
saya mau broadcast --> broadcast
saya mau broadcast --> broadcast
mau broadcast --> broadcast
broadcast --> broadcast
browser aktif --> browser
buka browser aktif --> browser
lihat jalan --> cctv_lewatmana
kondisi jalan --> cctv_lewatmana
saya mau lihat jalan --> cctv_lewatmana
bagaimana kondisi jalan --> cctv_lewatmana
lewat mana --> cctv_lewatmana
lihat cctv --> cctv_lewatmana
tampilkan cctv --> cctv_lewatmana
ada apa di jalan --> cctv_lewatmana
mana jalan yang macet --> cctv_lewatmana
ada kemacetan --> cctv_lewatma

## Proses step 2 - ambil sinonim

In [9]:
fls2 = glob.glob('./data/*/sinonim.txt')

In [10]:
pxx = []
pyy = []
for ff in fls2:
    with open(ff ,'r') as f:
        txx = f.readlines()
    for tx in txx:
        print(tx)
        tx = tx.split(':')

        for i in prep_x:
            if i.find(tx[0]) > -1:
                #lihat index dimana di temukan sinonim pertama
                ind = prep_x.index(i)
                for x in range(len(tx)):

                    #cari dari prep_x mana yg isinya seperti sinonim 1 -> lalu replace
                    px = prep_x[prep_x.index(i)].replace(tx[0],tx[x])
                    py = prep_y[prep_x.index(i)]
                    print(px,' ----> ',py )
                    pxx.append(px)
                    pyy.append(py)


broadcast:siaran
saya mau broadcast  ---->  broadcast
saya mau siaran  ---->  broadcast
saya mau broadcast  ---->  broadcast
saya mau siaran  ---->  broadcast
mau broadcast  ---->  broadcast
mau siaran  ---->  broadcast
broadcast  ---->  broadcast
siaran  ---->  broadcast
cctv:layar:kondisi
lihat cctv  ---->  cctv_lewatmana
lihat layar  ---->  cctv_lewatmana
lihat kondisi  ---->  cctv_lewatmana
tampilkan cctv  ---->  cctv_lewatmana
tampilkan layar  ---->  cctv_lewatmana
tampilkan kondisi  ---->  cctv_lewatmana
direktori:folder:map
buka direktori  ---->  direktori
buka folder  ---->  direktori
buka map  ---->  direktori
lihat direktori  ---->  direktori
lihat folder  ---->  direktori
lihat map  ---->  direktori
buka isi direktori  ---->  direktori
buka isi folder  ---->  direktori
buka isi map  ---->  direktori
lihat isi direktori  ---->  direktori
lihat isi folder  ---->  direktori
lihat isi map  ---->  direktori
google:web:browsing:info:informasi
buka google  ---->  google
buka web  -

In [11]:
print(len(pxx))
print(len(pyy))

310
310


### Dense representation one-hot encoding

In [12]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from numpy import argmax

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(intent)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
#print(onehot_encoded)

onehot_encoded[intent.index("goodbye")]
end_dense_layer = len(onehot_encoded)

C:\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [13]:
import pickle
pickle.dump(intent,open('i_intent.p','wb'))
pickle.dump(integer_encoded, open("i_integer_encoded.p","wb"))
pickle.dump(onehot_encoded, open("i_onehot_encoded.p",'wb'))

In [14]:
train_x=[]
train_y=[]
for i in range(len(prep_x)):
    train_x.append(swv_ar(twl(prep_x[i], rempunct=True, flat=True),frontpad=True))
    train_y.append(onehot_encoded[intent.index(prep_y[i])])

train_x=np.array(train_x)
train_y=np.array(train_y)

print(train_x.shape)
print(train_y.shape)

Problem at phrase: bobok
Problem at phrase: bobok
(168, 10, 300)
(168, 18)


### SIAPKAN DATA

In [15]:
#x_train, x_test, y_train, y_test = train_test_split(RQ, RA, test_size=0.2, random_state=1)

In [16]:
model=Sequential()
model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=False, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
model.add(Dense(75, activation="relu"))
model.add(Dense(end_dense_layer,activation="sigmoid"))
#model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=True, activation="relu", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
#model.add(LSTM(input_shape=(maxsent, num_features), return_sequences=True, activation="relu", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal"))
#adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#cosine_proximity
model.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 300)           721200    
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_1 (Dense)              (None, 75)                22575     
_________________________________________________________________
dense_2 (Dense)              (None, 18)                1368      
Total params: 1,466,343
Trainable params: 1,466,343
Non-trainable params: 0
_________________________________________________________________


### LOADING MODEL SUDAH DI TRAINING

In [18]:
#import keras
#model = keras.models.load_model("model1.chatbot")

# TRAINING MODEL

In [20]:
counter=0
while True:
    counter += 1
    print("EPOCH ===================== ",counter)
    result=model.fit(train_x,train_y, epochs=1)
    if result.history['acc'][0]>0.988:
        break

EPOCH =====================  1
Epoch 1/1
168/168 [==============================] - 0s 2ms/step - loss: -0.9917 - acc: 0.9881


In [21]:
    result=model.fit(train_x,train_y, epochs=5)


Epoch 1/5
168/168 [==============================] - 0s 2ms/step - loss: -0.9917 - acc: 0.9881
Epoch 2/5
168/168 [==============================] - 0s 2ms/step - loss: -0.9917 - acc: 0.9881
Epoch 3/5
168/168 [==============================] - 0s 2ms/step - loss: -0.9914 - acc: 0.9881
Epoch 4/5
168/168 [==============================] - 0s 2ms/step - loss: -0.9918 - acc: 0.9881
Epoch 5/5
168/168 [==============================] - 0s 2ms/step - loss: -0.9915 - acc: 0.9881


In [22]:
model.save("./model/model2.chatbot")